In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import pprint
pp = pprint.PrettyPrinter(indent=4)
%matplotlib inline

<h1>Example network: a Small World network</h1>

In [ ]:
sw = nx.watts_strogatz_graph(100,12,0.05)
# Initialize the position
pos = nx.spring_layout(sw,scale=0.2)
# draw all the nodes
nx.draw_networkx_nodes(sw,pos,node_size=500)
nx.draw_networkx_edges(sw,pos)
plt.show()

<h1>Community Detection</h1>

<h2>Algorithm: $k$-clique-community detection</h2>
$k$-clique detection finds communities of cliques of minimum degree $k$ that share at least $k-1$ nodes. The algorithm uses a percolation method to find these communities: a $k$-clique topological "template" is fitted over a clique, and one node of the clique is switched with another connected node outside of the clique. If this group of nodes is also a clique, that node is added to the $k$-clique-community. This is repeated until all cliques are visited.

In [ ]:
from networkx.algorithms.community import k_clique_communities

In [ ]:
for i in range(2,10):
    c = list(k_clique_communities(sw,i))
    print '{}-clique: '.format(i),len(c), "communities"

In [ ]:
# we use 6-clique communities
c = list(k_clique_communities(sw,6))
nx.draw_networkx(sw,pos)
for i in range(6):
    nx.draw_networkx_nodes(sw, pos, with_labels=True, nodelist=c[i], node_color=[np.random.uniform(0.25,0.95,3)]) # generates a random RGB color
nx.draw_networkx_edges(sw,pos)
plt.show()

<h2>Algorithm: Girvan-Newman</h2>
This method finds communities by iteratively removing edges with maximum betweenness centrality, recalculating centrality for the affected nodes, then removing the max-betweenness edge, etc. 

Betweenness centrality for edge $e$ is calculated as:
\begin{align}
g(e) = \sum_{s \neq t} \frac{\sigma_{st}(e)}{\sigma_{st}}
\end{align}
where $s,t$ are the starting and terminal nodes, and $\sigma_{st}$ is the number of shortest paths from $s\rightarrow t$, and $\sigma_{st}(e)$ is the number of those shortest paths that contain edge $e$

Determining when to stop removing edges is a free parameter in the algorithm. Typical choices include: reaching a predefined number of separate remaining subgraphs or maximization of modularity (discussed later). 

In [ ]:
from networkx.algorithms.community.centrality import girvan_newman
import itertools

In [ ]:
c = list(girvan_newman(sw))
k = 4
partitions =0;
comp = girvan_newman(sw)

limited = itertools.takewhile(lambda c: len(c) <= k, comp)
for communities in limited:
    partitions+=1
    print(tuple(sorted(c) for c in communities))
    
    
print 'Number of hierarchies:',partitions
print 'Selecting',k,'clusters:',c[partitions-1]
print 'First partition: ',c[0]
print 'Second partition: ',c[1]


In [ ]:
## Pick one with 4 partitions
cd = list(c[2])  
print len(cd)

In [ ]:
nx.draw_networkx(sw,pos)
for i in range(4):
    nx.draw_networkx_nodes(sw, pos, with_labels=True, nodelist=cd[i], node_color=[np.random.uniform(0.25,0.95,3)]) # generates a random RGB color
nx.draw_networkx_edges(sw,pos)
plt.show()

<h2>Metric: Modularity</h2>
Modularity is a network metric that measures how much community structure a network has. The modularity of a network calculated against a null random graph generated via the configuration model. The configuration model takes a prescribed degree distribution but has a random rewiring of edges. One can view the null configuration model graph as cutting all the edges of the original graph, but preserving the "stubs" on the vertices, and randomly rewiring all the stubs.

The modularity of a graph with $c$ communities is then calculated as:
\begin{align}
Q = \frac{1}{2m} \sum_{vw} \big[\frac{A_{vw}}{2m} - \frac{k_v k_w}{(2m)^2}\big]\delta(c_v, c_w)
\end{align}
where $m$ is the total number of edges, $v,w$ are nodes in the graph, $A_{vw}$ is the actual number of edges between $v$ and $w$, $k_v$ is the degree of the node, and $\delta$ is the dirac delta function.

The modularity thus measures how much more within-community connectivity we observe in the network over the null model (where connectivity is proportional simply to the degree of the node). By choosing which communities the vertices belong to, we can try to maximize $Q$.

<h2>Algorithm: Modularity Maximization Heuristics</h2>
(Louvain 2008) offers a percolation-inspired heuristic for community detection. The algorithm has two phases that are applied iteratively: the first phase is a greedy algorithm that places all nodes in their own community, then each node is moved to the community that gives the maximum gain in modularity. This is repeated for all nodes until no more modularity improvement can be made.

The second phase builds a new graph where the communities from phase 1 are collapsed into a single node, and the edges between communities collapse into single edges between the respective nodes with equivalent weight. Then phase 1 is repeated on this reduced graph, and the passes continue until no changes occur over a pass.

In [ ]:
!pip install python-louvain

In [ ]:
import community

#first compute the best partition
partition = community.best_partition(sw)

#drawing
size = float(len(set(partition.values())))
count = 0
for com in set(partition.values()) :
    count = count + 1
    list_nodes = [nodes for nodes in partition.keys()
                                if partition[nodes] == com]
    nx.draw_networkx_nodes(sw, pos, list_nodes,
                                node_color = np.random.uniform(0.25,0.95,3))

nx.draw_networkx_edges(sw, pos)
plt.show()

<h1>Centrality metrics</h1>

In [ ]:
# Degree centrality
dc = nx.degree_centrality(sw)
plt.figure(2, figsize=(7,7))
nx.draw(sw,
          pos,
          nodelist=dc.keys(),
          node_size = [d*7000 for d in dc.values()],
          node_color=dc.values(),
          font_size=8,
          cmap='spring',
          )

In [ ]:
# Closeness centrality
cl = nx.closeness_centrality(sw)
plt.figure(1, figsize=(7,7))
nx.draw(sw,
          pos,
          nodelist=cl.keys(),
          node_size = [d*3000 for d in cl.values()],
          node_color=cl.values(),
          font_size=8,
          cmap='summer',
          )

In [ ]:
# Betweenness centrality
bc = nx.betweenness_centrality(sw)
plt.figure(1, figsize=(7,7))
nx.draw(sw,
          pos,
          nodelist=bc.keys(),
          node_size = [d*10000 for d in bc.values()],
          node_color=bc.values(),
          font_size=8,
          cmap='autumn',
          )

In [ ]:
# Eigenvector centrality
ec = nx.eigenvector_centrality(sw ,max_iter=500)
plt.figure(1, figsize=(7,7))
nx.draw(sw,
          pos,
          nodelist=ec.keys(),
          node_size = [d*10000 for d in ec.values()],
          node_color=ec.values(),
          font_size=8,
          cmap='winter',
          )

In [ ]:
# plot two metrics against each other: closeness vs. eigenvector centrality
x = cl.values()
y = ec.values()
s = np.multiply(x,y)**2*1e5
plt.figure(1, figsize=(7,7))
plt.scatter(x,y,s = s)
plt.xlabel('Closeness Centrality')
plt.ylabel('Eigenvector Centrality')

In [ ]:
# list top 10 for both metrics
print 'Closeness Centrality Top 10:'
print sorted(cl, key=cl.get,reverse=True)[:10]
print 'Eigenvector Centrality Top 10:'
print sorted(ec, key=ec.get,reverse=True)[:10]